In [40]:
import numpy as np
import pandas as pd

In [41]:
DATA_SUFFIX = "-data.ssv"
ELEMENT_NAMES_SUFFIX = "-element-names.ens"


def get_raw_element_names(sample_name: str) -> list[str]:
    with open(f"{sample_name}{ELEMENT_NAMES_SUFFIX}", "r", encoding="utf-8") as f:
        return f.read().split(",")


def convert_raw_element_names_to_sv12(raw_element_names: list[str]) -> tuple[list[str]]:
    sv1 = [f"{name}_sv1" for name in raw_element_names]
    sv2 = [f"{name}_sv2" for name in raw_element_names]
    return (sv1, sv2,)


def generate_dtype(
        element_names_sv1: list[str],
        element_names_sv2: list[str]
        ) -> list[tuple[str]]:
    base_dt_types = [
        ("scheme_state_sv1", "bool"),
        ("scheme_state_sv2", "bool"),
        ("scheme_state", "bool"),
        ("probability", "float64")
    ]
    dt_types_sv1 = [(name, "bool") for name in element_names_sv1]
    dt_types_sv2 = [(name, "bool") for name in element_names_sv2]
    return [*base_dt_types, *dt_types_sv1, *dt_types_sv2]


def sum_probabilities(sample: pd.DataFrame) -> None:
    p = sample.loc[sample["scheme_state"] == True, "probability"].sum()
    q = sample.loc[sample["scheme_state"] == False, "probability"].sum()
    print(f"p = {p}")
    print(f"q = {q}")
    print(f"p + q = {p + q}")


def is_reconfiguration_same_or_better(sample: pd.DataFrame) -> bool:
    inconsistency = sample.loc[(sample["scheme_state_sv1"] == True) & (sample["scheme_state_sv2"] == False)]
    return inconsistency.empty


def sum_fail_probability_per_element(
        sample: pd.DataFrame,
        raw_element_names: list[str],
        element_names_sv1: list[str],
        element_names_sv2: list[str]
        ) -> None:
    failed_samples = sample.loc[sample["scheme_state"] == False]
    mask_sv1 = (failed_samples[element_names_sv1] == False)
    mask_sv2 = (failed_samples[element_names_sv2] == False)
    probabilities = failed_samples["probability"].values
    fail_probabilities_sv1 = np.sum(mask_sv1.values * probabilities[:, None], axis=0).tolist()
    fail_probabilities_sv2 = np.sum(mask_sv2.values * probabilities[:, None], axis=0).tolist()
    for i in range(len(raw_element_names)):
        print(f"{raw_element_names[i]}: sv1 = {fail_probabilities_sv1[i]:.8e}, sv2 = {fail_probabilities_sv2[i]:.8e}")


def sum_fail_count_per_element(
        sample: pd.DataFrame,
        raw_element_names: list[str],
        element_names_sv1: list[str],
        element_names_sv2: list[str]
        ) -> None:
    failed_samples_sv1 = sample.loc[sample["scheme_state"] == False]
    fail_count_sv1 = (failed_samples_sv1[element_names_sv1] == False).sum()
    failed_samples_sv2 = sample.loc[sample["scheme_state"] == False]
    fail_count_sv2 = (failed_samples_sv2[element_names_sv2] == False).sum()
    for i in range(len(raw_element_names)):
        print(f"{raw_element_names[i]}: sv1 = {fail_count_sv1.iloc[i]}, sv2 = {fail_count_sv2.iloc[i]}")


def process_sample(sample_name: str) -> tuple[list[str], list[str], list[str], pd.DataFrame]:
    raw_element_names = get_raw_element_names(sample_name)
    element_names_sv1, element_names_sv2 = convert_raw_element_names_to_sv12(raw_element_names)
    dtype = generate_dtype(element_names_sv1, element_names_sv2)
    samples = pd.DataFrame(np.fromfile(f"{sample_name}{DATA_SUFFIX}", dtype=dtype))

    print("=== sum probabilities ===")
    sum_probabilities(samples)

    if is_reconfiguration_same_or_better(samples):
        print("=== reconfiguration is consistent ===")
    else:
        print("=== reconfiguration is inconsistent ===")
        return

    print("=== sum fail probability(sv1 | sv2) per element ===")
    sum_fail_probability_per_element(
        samples,
        raw_element_names,
        element_names_sv1,
        element_names_sv2
    )

    print("=== sum fail count(sv1 | sv2) per element ===")
    sum_fail_count_per_element(
        samples,
        raw_element_names,
        element_names_sv1,
        element_names_sv2
    )

    return (raw_element_names, element_names_sv1, element_names_sv2, samples)

In [42]:
def s23_original_brute():
    raw_element_names, element_names_sv1, element_names_sv2, samples = process_sample("s23-original-brute")

    pr_cases_sv1 = (
        samples.loc[(samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True)],
        samples.loc[(samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == False)],
        samples.loc[(samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == False) & (samples["pr3_sv1"] == True)],
        samples.loc[(samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == False) & (samples["pr3_sv1"] == False)],
        samples.loc[(samples["pr1_sv1"] == False) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True)],
        samples.loc[(samples["pr1_sv1"] == False) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == False)],
        samples.loc[(samples["pr1_sv1"] == False) & (samples["pr2_sv1"] == False) & (samples["pr3_sv1"] == True)],
        samples.loc[(samples["pr1_sv1"] == False) & (samples["pr2_sv1"] == False) & (samples["pr3_sv1"] == False)]
    )

    pr_cases_sv2 = (
        samples.loc[(samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True)],
        samples.loc[(samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == False)],
        samples.loc[(samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == False) & (samples["pr3_sv2"] == True)],
        samples.loc[(samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == False) & (samples["pr3_sv2"] == False)],
        samples.loc[(samples["pr1_sv2"] == False) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True)],
        samples.loc[(samples["pr1_sv2"] == False) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == False)],
        samples.loc[(samples["pr1_sv2"] == False) & (samples["pr2_sv2"] == False) & (samples["pr3_sv2"] == True)],
        samples.loc[(samples["pr1_sv2"] == False) & (samples["pr2_sv2"] == False) & (samples["pr3_sv2"] == False)]
    )

    print("=== sum fail count(sv1 | sv2) per pr1 * pr2 * pr3 state set ===")
    for i in range(8):
        print(f"case {i}: sv1 = {pr_cases_sv1[i].shape[0]}; sv2 = {pr_cases_sv2[i].shape[0]}")

s23_original_brute()

FileNotFoundError: [Errno 2] No such file or directory: 's23-original-brute-element-names.ens'

In [ ]:
def s23_original_greedy():
    raw_element_names, element_names_sv1, element_names_sv2, samples = process_sample("s23-original-greedy")

    pr_cases_sv1 = (
        samples.loc[(samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True)],
        samples.loc[(samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == False)],
        samples.loc[(samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == False) & (samples["pr3_sv1"] == True)],
        samples.loc[(samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == False) & (samples["pr3_sv1"] == False)],
        samples.loc[(samples["pr1_sv1"] == False) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True)],
        samples.loc[(samples["pr1_sv1"] == False) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == False)],
        samples.loc[(samples["pr1_sv1"] == False) & (samples["pr2_sv1"] == False) & (samples["pr3_sv1"] == True)],
        samples.loc[(samples["pr1_sv1"] == False) & (samples["pr2_sv1"] == False) & (samples["pr3_sv1"] == False)]
    )

    pr_cases_sv2 = (
        samples.loc[(samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True)],
        samples.loc[(samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == False)],
        samples.loc[(samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == False) & (samples["pr3_sv2"] == True)],
        samples.loc[(samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == False) & (samples["pr3_sv2"] == False)],
        samples.loc[(samples["pr1_sv2"] == False) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True)],
        samples.loc[(samples["pr1_sv2"] == False) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == False)],
        samples.loc[(samples["pr1_sv2"] == False) & (samples["pr2_sv2"] == False) & (samples["pr3_sv2"] == True)],
        samples.loc[(samples["pr1_sv2"] == False) & (samples["pr2_sv2"] == False) & (samples["pr3_sv2"] == False)]
    )

    print("=== sum fail count(sv1 | sv2) per pr1 * pr2 * pr3 state set ===")
    for i in range(8):
        print(f"case {i}: sv1 = {pr_cases_sv1[i].shape[0]}; sv2 = {pr_cases_sv2[i].shape[0]}")

s23_original_greedy()

=== sum probabilities ===
p = 0.9982019946444761
q = 0.001798005355523381
p + q = 0.9999999999999996
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 2.44501962e-07, sv2 = 4.31965440e-08
pr2: sv1 = 2.44501962e-07, sv2 = 4.31965440e-08
pr3: sv1 = 2.44501962e-07, sv2 = 4.31965440e-08
pr5: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
pr6: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a1: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a2: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
b1: sv1 = 2.69700803e-08, sv2 = 2.69700803e-08
b2: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
b4: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
b5: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
c1: sv1 = 4.10000000e-04, sv2 = 4.10000000e-04
c2: sv1 = 4.10000000e-04, sv2 = 4.10000000e-04
c4: sv1 = 4.10000000e-04, sv2 = 4.10000000e-04
c5: sv1 = 9.04911182e-07, sv2 = 9.04911182e-07
c6: sv1 = 9.04911182e-07, sv2 = 9.04911182e-07
d1: sv1 = 4.00392263e-08, sv2 = 4.00392263e-08
d2: s

In [47]:
def s23_77788_greedy():
    raw_element_names, element_names_sv1, element_names_sv2, samples = process_sample("s23-77788-greedy")

    fail_samples_sv1 = samples.loc[samples["scheme_state_sv1"] == False]
    pr_cases_sv1 = (
        fail_samples_sv1.loc[(fail_samples_sv1["pr1_sv1"] == True) & (fail_samples_sv1["pr2_sv1"] == True) & (fail_samples_sv1["pr3_sv1"] == True)],
        fail_samples_sv1.loc[(fail_samples_sv1["pr1_sv1"] == True) & (fail_samples_sv1["pr2_sv1"] == True) & (fail_samples_sv1["pr3_sv1"] == False)],
        fail_samples_sv1.loc[(fail_samples_sv1["pr1_sv1"] == True) & (fail_samples_sv1["pr2_sv1"] == False) & (fail_samples_sv1["pr3_sv1"] == True)],
        fail_samples_sv1.loc[(fail_samples_sv1["pr1_sv1"] == True) & (fail_samples_sv1["pr2_sv1"] == False) & (fail_samples_sv1["pr3_sv1"] == False)],
        fail_samples_sv1.loc[(fail_samples_sv1["pr1_sv1"] == False) & (fail_samples_sv1["pr2_sv1"] == True) & (fail_samples_sv1["pr3_sv1"] == True)],
        fail_samples_sv1.loc[(fail_samples_sv1["pr1_sv1"] == False) & (fail_samples_sv1["pr2_sv1"] == True) & (fail_samples_sv1["pr3_sv1"] == False)],
        fail_samples_sv1.loc[(fail_samples_sv1["pr1_sv1"] == False) & (fail_samples_sv1["pr2_sv1"] == False) & (fail_samples_sv1["pr3_sv1"] == True)],
        fail_samples_sv1.loc[(fail_samples_sv1["pr1_sv1"] == False) & (fail_samples_sv1["pr2_sv1"] == False) & (fail_samples_sv1["pr3_sv1"] == False)]
    )

    fail_samples_sv2 = samples.loc[samples["scheme_state_sv2"] == False]
    pr_cases_sv2 = (
        fail_samples_sv2.loc[(fail_samples_sv2["pr1_sv2"] == True) & (fail_samples_sv2["pr2_sv2"] == True) & (fail_samples_sv2["pr3_sv2"] == True)],
        fail_samples_sv2.loc[(fail_samples_sv2["pr1_sv2"] == True) & (fail_samples_sv2["pr2_sv2"] == True) & (fail_samples_sv2["pr3_sv2"] == False)],
        fail_samples_sv2.loc[(fail_samples_sv2["pr1_sv2"] == True) & (fail_samples_sv2["pr2_sv2"] == False) & (fail_samples_sv2["pr3_sv2"] == True)],
        fail_samples_sv2.loc[(fail_samples_sv2["pr1_sv2"] == True) & (fail_samples_sv2["pr2_sv2"] == False) & (fail_samples_sv2["pr3_sv2"] == False)],
        fail_samples_sv2.loc[(fail_samples_sv2["pr1_sv2"] == False) & (fail_samples_sv2["pr2_sv2"] == True) & (fail_samples_sv2["pr3_sv2"] == True)],
        fail_samples_sv2.loc[(fail_samples_sv2["pr1_sv2"] == False) & (fail_samples_sv2["pr2_sv2"] == True) & (fail_samples_sv2["pr3_sv2"] == False)],
        fail_samples_sv2.loc[(fail_samples_sv2["pr1_sv2"] == False) & (fail_samples_sv2["pr2_sv2"] == False) & (fail_samples_sv2["pr3_sv2"] == True)],
        fail_samples_sv2.loc[(fail_samples_sv2["pr1_sv2"] == False) & (fail_samples_sv2["pr2_sv2"] == False) & (fail_samples_sv2["pr3_sv2"] == False)]
    )

    print("=== sum fail count(sv1 | sv2) per pr1 * pr2 * pr3 state set ===")
    for i in range(8):
        print(f"case {i}: sv1 = {pr_cases_sv1[i].shape[0]}; sv2 = {pr_cases_sv2[i].shape[0]}")

s23_77788_greedy()

=== sum probabilities ===
p = 0.9984416062472431
q = 0.0015583937527566018
p + q = 0.9999999999999997
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 2.15757193e-07, sv2 = 2.88069101e-08
pr2: sv1 = 2.15757193e-07, sv2 = 2.88069101e-08
pr3: sv1 = 2.15757193e-07, sv2 = 2.88069101e-08
pr5: sv1 = 1.87012424e-07, sv2 = 1.55488898e-11
pr6: sv1 = 1.87012424e-07, sv2 = 4.31999988e-08
a1: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a2: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
b1: sv1 = 2.33759063e-08, sv2 = 2.33759063e-08
b2: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
b4: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
b5: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
c1: sv1 = 4.10000000e-04, sv2 = 4.10000000e-04
c2: sv1 = 4.10000000e-04, sv2 = 4.10000000e-04
c4: sv1 = 4.10000000e-04, sv2 = 4.10000000e-04
c5: sv1 = 8.06710687e-07, sv2 = 8.06710687e-07
c6: sv1 = 8.06710687e-07, sv2 = 8.06710687e-07
d1: sv1 = 3.47678870e-08, sv2 = 3.47678870e-08
d2: 

In [44]:
def s23_77788_brute():
    raw_element_names, element_names_sv1, element_names_sv2, samples = process_sample("s23-77788-brute")

    fail_samples_sv1 = samples.loc[samples["scheme_state_sv1"] == False]
    pr_cases_sv1 = (
        fail_samples_sv1.loc[(fail_samples_sv1["pr1_sv1"] == True) & (fail_samples_sv1["pr2_sv1"] == True) & (fail_samples_sv1["pr3_sv1"] == True)],
        fail_samples_sv1.loc[(fail_samples_sv1["pr1_sv1"] == True) & (fail_samples_sv1["pr2_sv1"] == True) & (fail_samples_sv1["pr3_sv1"] == False)],
        fail_samples_sv1.loc[(fail_samples_sv1["pr1_sv1"] == True) & (fail_samples_sv1["pr2_sv1"] == False) & (fail_samples_sv1["pr3_sv1"] == True)],
        fail_samples_sv1.loc[(fail_samples_sv1["pr1_sv1"] == True) & (fail_samples_sv1["pr2_sv1"] == False) & (fail_samples_sv1["pr3_sv1"] == False)],
        fail_samples_sv1.loc[(fail_samples_sv1["pr1_sv1"] == False) & (fail_samples_sv1["pr2_sv1"] == True) & (fail_samples_sv1["pr3_sv1"] == True)],
        fail_samples_sv1.loc[(fail_samples_sv1["pr1_sv1"] == False) & (fail_samples_sv1["pr2_sv1"] == True) & (fail_samples_sv1["pr3_sv1"] == False)],
        fail_samples_sv1.loc[(fail_samples_sv1["pr1_sv1"] == False) & (fail_samples_sv1["pr2_sv1"] == False) & (fail_samples_sv1["pr3_sv1"] == True)],
        fail_samples_sv1.loc[(fail_samples_sv1["pr1_sv1"] == False) & (fail_samples_sv1["pr2_sv1"] == False) & (fail_samples_sv1["pr3_sv1"] == False)]
    )

    fail_samples_sv2 = samples.loc[samples["scheme_state_sv2"] == False]
    pr_cases_sv2 = (
        fail_samples_sv2.loc[(fail_samples_sv2["pr1_sv2"] == True) & (fail_samples_sv2["pr2_sv2"] == True) & (fail_samples_sv2["pr3_sv2"] == True)],
        fail_samples_sv2.loc[(fail_samples_sv2["pr1_sv2"] == True) & (fail_samples_sv2["pr2_sv2"] == True) & (fail_samples_sv2["pr3_sv2"] == False)],
        fail_samples_sv2.loc[(fail_samples_sv2["pr1_sv2"] == True) & (fail_samples_sv2["pr2_sv2"] == False) & (fail_samples_sv2["pr3_sv2"] == True)],
        fail_samples_sv2.loc[(fail_samples_sv2["pr1_sv2"] == True) & (fail_samples_sv2["pr2_sv2"] == False) & (fail_samples_sv2["pr3_sv2"] == False)],
        fail_samples_sv2.loc[(fail_samples_sv2["pr1_sv2"] == False) & (fail_samples_sv2["pr2_sv2"] == True) & (fail_samples_sv2["pr3_sv2"] == True)],
        fail_samples_sv2.loc[(fail_samples_sv2["pr1_sv2"] == False) & (fail_samples_sv2["pr2_sv2"] == True) & (fail_samples_sv2["pr3_sv2"] == False)],
        fail_samples_sv2.loc[(fail_samples_sv2["pr1_sv2"] == False) & (fail_samples_sv2["pr2_sv2"] == False) & (fail_samples_sv2["pr3_sv2"] == True)],
        fail_samples_sv2.loc[(fail_samples_sv2["pr1_sv2"] == False) & (fail_samples_sv2["pr2_sv2"] == False) & (fail_samples_sv2["pr3_sv2"] == False)]
    )

    print("=== sum fail count(sv1 | sv2) per pr1 * pr2 * pr3 state set ===")
    for i in range(8):
        print(f"case {i}: sv1 = {pr_cases_sv1[i].shape[0]}; sv2 = {pr_cases_sv2[i].shape[0]}")

s23_77788_brute()

=== sum probabilities ===
p = 0.9984416493643964
q = 0.0015583506356032617
p + q = 0.9999999999999997
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 1.87012424e-07, sv2 = 1.72800000e-12
pr2: sv1 = 1.87012424e-07, sv2 = 8.63875589e-12
pr3: sv1 = 1.87012424e-07, sv2 = 5.18337794e-12
pr5: sv1 = 1.87012424e-07, sv2 = 6.22030234e-16
pr6: sv1 = 1.87012424e-07, sv2 = 5.18337794e-12
a1: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a2: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
b1: sv1 = 2.33752595e-08, sv2 = 2.33752595e-08
b2: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
b4: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
b5: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
c1: sv1 = 4.10000000e-04, sv2 = 4.10000000e-04
c2: sv1 = 4.10000000e-04, sv2 = 4.10000000e-04
c4: sv1 = 4.10000000e-04, sv2 = 4.10000000e-04
c5: sv1 = 8.06693016e-07, sv2 = 8.06693016e-07
c6: sv1 = 8.06693016e-07, sv2 = 8.06693016e-07
d1: sv1 = 3.47669385e-08, sv2 = 3.47669385e-08
d2: 